Надо будет сгруппировать по длительности гласных по сегам, которые отправят.

Надо организовать короч так.
В первой колонке слоги, дальше длина слога, а затем позиция слога.

Нужно еще позицию в слове учитывать. И шкалирование из KMEans заюзать.


НАДО СДЕЛАТЬ НОРМАЛИЗАЦИЮ - НЕ ЗАБЫТЬ!!!

In [ ]:
from PIL import Image, ImageDraw, ImageFont
from glob import glob
import pandas as pd

In [ ]:
def readfile(filename):
  lines=[line for line in open(filename)]
  
  colnames=lines[0].strip().split('\t')[1:]
  rownames=[]
  data=[]
  for line in lines[1:]:
    p=line.strip().split('\t')

    rownames.append(p[0])

    data.append([float(x) for x in p[1:]])
  return rownames,colnames,data


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path = "/content/drive/MyDrive/v_models/cta0001-0010"

In [ ]:
seg_files_b1 = glob(path+'/*.seg_B1')

##Вот здесь я формирую выборку для обучения иерархической кластеризации

In [ ]:
def readsegs(file_names):
  syllables = []
  for f_n in file_names:
    lines=[line.strip() for line in open(f_n)]
    syl_line = []
    syllable = ''
    length = 0
    position = 0
    for i in range(len(lines[7:])):
      k = i + 7
      labels = lines[k].split(",")
      if labels[2] != "":
        vowels = ['a','e','i','o','u','y']
        v_bool = False
        for v in vowels:
          if v in labels[2]:
            v_bool = True
        if not v_bool:
          syllable = syllable + labels[2]
          length += int(lines[k + 1].split(",")[0]) - int(labels[0])
        if v_bool:
          syllable = syllable + labels[2][0]
          position = int(labels[2][1])
          length = int(lines[k + 1].split(",")[0]) - int(labels[0])
          if position == 4:
            position = -1
          length = round(int(length / int(lines[2].split('=')[1]) / int(lines[4].split('=')[1])), 2)
          syl_line += [[syllable, length, int(position)]]
          syllable = ""
          position = 0
          v_bool = False
    syllables += syl_line
  

  for isyl in range(len(syllables)):
    z = 1
    g = -1
    if syllables[isyl][2] == 1:
      for i in range(len(syllables[isyl + 1:])):
        if syllables[i + isyl + 1][2] > 0:
          z += 1
        else:
          break
      syllables[isyl][2] = z
      z = 1
    
    if syllables[isyl][2] == -1:
      for i in range(len(syllables[:isyl]), 0, -1):
        if syllables[i - 1][2] < 0:
          g -= 1
        else:
          break
      syllables[isyl][2] = g
      g = -1
  res = ['Syllable\tlength\tposition']
  for i in syllables:
    g = [str(l) for l in i]
    res.append("\t".join(g))
  with open('syllables.txt', 'w') as fw:
    fw.write("\n".join(res))
  return res

In [ ]:
readsegs(seg_files_b1)

['Syllable\tlength\tposition',
 'a\t3159\t1',
 "bm'e\t3768\t0",
 'vy\t1807\t1',
 'ju\t2148\t0',
 "l'i\t1534\t-1",
 'ma\t2587\t0',
 "d'dm'i\t1676\t0",
 "tr'i\t1284\t-1",
 'i\t1376\t-2',
 'va\t1107\t-3',
 "ks'e\t2165\t0",
 "n'i\t1227\t-1",
 'e\t1503\t-2',
 "f'o\t1955\t0",
 'dy\t701\t-1',
 'ro\t1410\t-2',
 'vna\t4132\t-3',
 "t'i\t1759\t2",
 'zhy\t1434\t1',
 'lo\t1838\t0',
 'zy\t1043\t2',
 'ba\t1934\t1',
 "l'e\t3109\t0",
 'la\t2565\t-1',
 'i\t3404\t2',
 'i\t2254\t1',
 'jo\t1638\t0',
 'a\t1722\t2',
 "tv'i\t1519\t1",
 "zl'i\t1714\t0",
 'vbo\t2604\t0',
 "tk'i\t1376\t-1",
 'nsku\t1437\t-2',
 'ju\t2562\t-3',
 "gd'e\t1235\t0",
 'a\t1534\t1',
 'na\t1554\t0',
 'pra\t708\t2',
 "l'i\t1448\t1",
 'zha\t2156\t0',
 'la\t1221\t-1',
 "dv'i\t1147\t1",
 'na\t2077\t0',
 'cy\t1181\t-1',
 "d'dn'e\t2276\t0",
 'jspa\t1013\t2',
 'da\t1999\t1',
 "zr'e\t2106\t0",
 "n'i\t718\t-1",
 'e\t820\t-2',
 'mna\t1648\t1',
 'sa\t2051\t0',
 'mo\t1422\t-1',
 'e\t1322\t-2',
 'hu\t1926\t0',
 'tshy\t1102\t-1',
 'e\t3024\t-2',
 'ry\

In [ ]:
from math import sqrt

def pearson(v1,v2):

  sum1=sum(v1)
  sum2=sum(v2)

  sum1Sq=sum([pow(v,2) for v in v1])
  sum2Sq=sum([pow(v,2) for v in v2])	
  
  # Sum of the products

  pSum=sum([v1[i]*v2[i] for i in range(len(v1))])
  
  # Calculate r (Pearson score)
  num=pSum-(sum1*sum2/len(v1))
  den=sqrt((sum1Sq-pow(sum1,2)/len(v1))*(sum2Sq-pow(sum2,2)/len(v1)))

  if den==0: return 0

  return 1.0-num/den


In [ ]:
import numpy as np

from scipy.stats import pearsonr

pearsonr([3159.0, 1.0], [3768.0, 0.0])[0]

1.0

In [ ]:
class bicluster:
  def __init__(self,vec,left=None,right=None,distance=0.0,id=None):
    self.left=left
    self.right=right
    self.vec=vec # вектор признаков
    self.id=id # id объединения
    self.distance=distance # степень схожести между левым и правым


In [ ]:
def hcluster(rows,distance=pearson):
  distances={}
  currentclustid=-1

  clust=[bicluster(rows[i],id=i) for i in range(len(rows))]

  while len(clust)>1:
    lowestpair=(0,1)
    closest=distance(clust[0].vec,clust[1].vec)
    # loop through every pair looking for the smallest distance

    for i in range(len(clust)):
      for j in range(i+1,len(clust)):
        # distances is the cache of distance calculations
        if (clust[i].id,clust[j].id) not in distances: 
          distances[(clust[i].id,clust[j].id)]=distance(clust[i].vec,clust[j].vec)

        d=distances[(clust[i].id,clust[j].id)]
        if d<closest:
          closest=d
          lowestpair=(i,j)
    # calculate the average of the two clusters
    mergevec=[
      (clust[lowestpair[0]].vec[i]+clust[lowestpair[1]].vec[i])/2.0 
        for i in range(len(clust[0].vec))
    ]
    # create the new cluster
    newcluster=bicluster(
        mergevec,
        left=clust[lowestpair[0]],
        right=clust[lowestpair[1]],
        distance=closest,id=currentclustid
    )
    # cluster ids that weren't in the original set are negative
    currentclustid-=1
    del clust[lowestpair[1]]
    del clust[lowestpair[0]]
    clust.append(newcluster)
  return clust[0]


In [1]:
def printclust(clust,labels=None,n=0):
  # indent to make a hierarchy layout

  for i in range(n): print(' ')
  if clust.id<0:
    # negative id means that this is branch
    print('-')
  else:
    # positive id means that this is an endpoint
    if labels==None: print(clust.id)
    else: print(labels[clust.id])
  # now print the right and left branches
  if clust.left!=None: printclust(clust.left,labels=labels,n=n+1)
  if clust.right!=None: printclust(clust.right,labels=labels,n=n+1)


In [ ]:
syllables,features,data=readfile('syllables.txt')

In [ ]:
print(data)

[[3159.0, 1.0], [3768.0, 0.0], [1807.0, 1.0], [2148.0, 0.0], [1534.0, -1.0], [2587.0, 0.0], [1676.0, 0.0], [1284.0, -1.0], [1376.0, -2.0], [1107.0, -3.0], [2165.0, 0.0], [1227.0, -1.0], [1503.0, -2.0], [1955.0, 0.0], [701.0, -1.0], [1410.0, -2.0], [4132.0, -3.0], [1759.0, 2.0], [1434.0, 1.0], [1838.0, 0.0], [1043.0, 2.0], [1934.0, 1.0], [3109.0, 0.0], [2565.0, -1.0], [3404.0, 2.0], [2254.0, 1.0], [1638.0, 0.0], [1722.0, 2.0], [1519.0, 1.0], [1714.0, 0.0], [2604.0, 0.0], [1376.0, -1.0], [1437.0, -2.0], [2562.0, -3.0], [1235.0, 0.0], [1534.0, 1.0], [1554.0, 0.0], [708.0, 2.0], [1448.0, 1.0], [2156.0, 0.0], [1221.0, -1.0], [1147.0, 1.0], [2077.0, 0.0], [1181.0, -1.0], [2276.0, 0.0], [1013.0, 2.0], [1999.0, 1.0], [2106.0, 0.0], [718.0, -1.0], [820.0, -2.0], [1648.0, 1.0], [2051.0, 0.0], [1422.0, -1.0], [1322.0, -2.0], [1926.0, 0.0], [1102.0, -1.0], [3024.0, -2.0], [1392.0, 2.0], [1963.0, 1.0], [1673.0, 0.0], [1487.0, -1.0], [1288.0, 1.0], [1739.0, 0.0], [1013.0, 0.0], [1389.0, -1.0], [1320

## Округляю полученные значения

In [ ]:
df = pd.DataFrame(data)

In [ ]:
df.head()

,0,1
0,3159.0,1.0
1,3768.0,0.0
2,1807.0,1.0
3,2148.0,0.0
4,1534.0,-1.0


In [ ]:
df = (df-df.min ())/(df.max ()-df.min ())

In [ ]:
df = df.round(2)

In [ ]:
df.head()

,0,1
0,0.53,0.67
1,0.64,0.50
2,0.28,0.67
3,0.34,0.50
4,0.22,0.33


In [ ]:
data = list(df.itertuples(index=False, name=None))

In [ ]:
clust=hcluster(data)

In [ ]:
def getheight(clust):
  if clust.left==None and clust.right==None: return 1
  return getheight(clust.left)+getheight(clust.right)

def getdepth(clust):
  if clust.left==None and clust.right==None: return 0
  return max(getdepth(clust.left),getdepth(clust.right))+clust.distance


##Хм, ерунда какая-то. Он все равно разбил значения только на 2 кластера, хотя они нормализованы и даже округлены!

In [ ]:
print(getdepth(clust))
print(getheight(clust))

2.000000000000016
403


In [2]:
printclust(clust, labels=syllables)

NameError: ignored

In [ ]:
def drawdendrogram(clust,labels,jpeg='clusters.jpg'):
  h=getheight(clust)*20
  w=1200
  depth=getdepth(clust)
  if depth != 0:
    scaling=float(w-150)/depth
  else:
    scaling=float(w-150)
  img=Image.new('RGB',(w,h),(255,255,255))
  draw=ImageDraw.Draw(img)

  draw.line((0,h/2,10,h/2),fill=(255,0,0))    
  drawnode(draw,clust,10,(h/2),scaling,labels)
  
  img.save(jpeg,'JPEG')


In [ ]:
def drawnode(draw,clust,x,y,scaling,labels):

  if clust.id<0:

    h1=getheight(clust.left)*20

    h2=getheight(clust.right)*20

    top=y-(h1+h2)/2

    bottom=y+(h1+h2)/2

    ll=clust.distance*scaling

    draw.line((x,top+h1/2,x,bottom-h2/2),fill=(255,0,0))    

    draw.line((x,top+h1/2,x+ll,top+h1/2),fill=(255,0,0))    

    draw.line((x,bottom-h2/2,x+ll,bottom-h2/2),fill=(255,0,0))        

    drawnode(draw,clust.left,x+ll,top+h1/2,scaling,labels)
    drawnode(draw,clust.right,x+ll,bottom-h2/2,scaling,labels)
  else:   
    draw.text((x+5,y-7),labels[clust.id],(0,0,0))


In [ ]:
drawdendrogram(clust,syllables,jpeg='syllablesclust.jpg')
